In [1]:
import os
import tensorflow as tf
from tensorflow.keras import backend
from sagemaker.tensorflow import TensorFlowModel


## Preparing model.tar.gz for deploying to Sagemaker Endpoint
Pretrained YoloV5 models can be accessed through https://github.com/ultralytics/yolov5

First, `git clone https://github.com/ultralytics/yolov5` to your `ROOT` directory, then within the `ROOT/yolov5` directory, you can retrieve a pretrained yolov5 model by running the following command: `python export.py --weights yolov5l.pt --include saved_model --nms`. This will create a directory called `yolov5l_saved_model`. 

In order to deploy this model onto SageMaker Endpoints, it must be packaged in the following structure:

```
export
└──Servo
   └──1
      │  saved_model.pb
      └──assets
      └──variables
         │ variables.data-00000-of-00001
         │ variables.index
```

To get the correct data structure for `model.tar.gz`, copy the contents from `yolov5l_saved_model` into `export/Servo/1/`. Once you have the correct, directory structure for your model, use `tar -czvf model.tar.gz export` to get your `model.tar.gz`.

Upload `model.tar.gz` to your s3 bucket of your choice, and make sure your ec2 instance or sagemaker instance has the correct IAM roles to access the s3 bucket where `model.tar.gz` will be located. 

In my case, my `model.tar.gz` will be uploaded to `s3://sagemaker-us-east-1-171547146718/model.tar.gz`


In [4]:
!tar -czvf model.tar.gz export


/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
export/
export/Servo/
export/Servo/1/
export/Servo/1/.ipynb_checkpoints/
export/Servo/1/variables/
export/Servo/1/variables/variables.index
export/Servo/1/variables/variables.data-00000-of-00001
export/Servo/1/assets/
export/Servo/1/saved_model.pb


In [2]:
model_data = 's3://<s3 bucket>/model.tar.gz'
role = '<IAM ROLE>'

In [11]:
model = TensorFlowModel(model_data=model_data, 
                        framework_version='2.3', role=role)
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m4.xlarge',
                         #instance_type='ml.g4dn.xlarge',#'ml.m4.xlarge',
                         #accelerator_type='ml.eia2.medium',
                         endpoint_name='checkride-demo-yolov5l')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


--------!

That's it! Now you're ready to invoke the sagemaker endpoint to get predictions back